In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import pickle

from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator
from diffusers import (
    PNDMScheduler,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    KDPM2DiscreteScheduler,
    DEISMultistepScheduler,
)

from pts.model.time_grad import TimeGradEstimator
from pts.dataset.repository.datasets import dataset_recipes

from utils import omit_points, interpolate_np_array

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/artificial/_base.py:84: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.freq: BaseOffset = to_offset(freq)


In [3]:
dataset = get_dataset("electricity_nips", regenerate=False)

train_grouper = MultivariateGrouper(
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality)
)

test_grouper = MultivariateGrouper(
    num_test_dates=int(len(dataset.test) / len(dataset.train)),
    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
)

dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/dataset/common.py:262: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.Period(val, freq)


In [4]:
evaluator = MultivariateEvaluator(
    quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={"sum": np.sum}
)

In [5]:
def plot(
    target,
    forecast,
    prediction_length,
    prediction_intervals=(50.0, 90.0),
    color="g",
    fname=None,
):
    label_prefix = ""
    rows = 4
    cols = 4
    fig, axs = plt.subplots(rows, cols, figsize=(24, 24))
    axx = axs.ravel()
    seq_len, target_dim = target.shape

    ps = [50.0] + [
        50.0 + f * c / 2.0 for c in prediction_intervals for f in [-1.0, +1.0]
    ]

    percentiles_sorted = sorted(set(ps))

    def alpha_for_percentile(p):
        return (p / 100.0) ** 0.3

    for dim in range(0, min(rows * cols, target_dim)):
        ax = axx[dim]

        target[-2 * prediction_length :][dim].plot(ax=ax)

        ps_data = [forecast.quantile(p / 100.0)[:, dim] for p in percentiles_sorted]
        i_p50 = len(percentiles_sorted) // 2

        p50_data = ps_data[i_p50]
        p50_series = pd.Series(data=p50_data, index=forecast.index)
        p50_series.plot(color=color, ls="-", label=f"{label_prefix}median", ax=ax)

        for i in range(len(percentiles_sorted) // 2):
            ptile = percentiles_sorted[i]
            alpha = alpha_for_percentile(ptile)
            ax.fill_between(
                forecast.index,
                ps_data[i],
                ps_data[-i - 1],
                facecolor=color,
                alpha=alpha,
                interpolate=True,
            )
            # Hack to create labels for the error intervals.
            # Doesn't actually plot anything, because we only pass a single data point
            pd.Series(data=p50_data[:1], index=forecast.index[:1]).plot(
                color=color,
                alpha=alpha,
                linewidth=10,
                label=f"{label_prefix}{100 - ptile * 2}%",
                ax=ax,
            )

    legend = ["observations", "median prediction"] + [
        f"{k}% prediction interval" for k in prediction_intervals
    ][::-1]
    axx[0].legend(legend, loc="upper left")

    if fname is not None:
        plt.savefig(fname, bbox_inches="tight", pad_inches=0.05)


## Train TimeGrad on Electricity dataset

In [6]:
scheduler = DEISMultistepScheduler(
    num_train_timesteps=150,
    beta_end=0.1,
)

In [7]:
estimator = TimeGradEstimator(
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    hidden_size=64,
    num_layers=2,
    dropout_rate=0.1,
    lags_seq=[1],
    scheduler=scheduler,
    num_inference_steps=149,
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="mean",
    trainer_kwargs=dict(max_epochs=200, accelerator="gpu", devices="1")
)

/home/jupyter/.local/lib/python3.10/site-packages/gluonts/time_feature/_base.py:243: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)


In [8]:
predictor = estimator.train(dataset_train, cache_data=True, shuffle_buffer_length=1024)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jupyter/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-05-22 08:45:38.063288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 08:45:39.223474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params | In sizes                                                             | Out sizes        
-------------------------------------------------------

Epoch 0: |          | 50/? [00:06<00:00,  8.02it/s, v_num=14, train_loss=0.338]

Epoch 0, global step 50: 'train_loss' reached 0.33825 (best 0.33825), saving model to '/home/jupyter/work/resources/lightning_logs/version_14/checkpoints/epoch=0-step=50.ckpt' as top 1


Epoch 1: |          | 50/? [00:06<00:00,  8.15it/s, v_num=14, train_loss=0.0982]

Epoch 1, global step 100: 'train_loss' reached 0.09817 (best 0.09817), saving model to '/home/jupyter/work/resources/lightning_logs/version_14/checkpoints/epoch=1-step=100.ckpt' as top 1


...

Epoch 198: |          | 50/? [00:06<00:00,  8.28it/s, v_num=14, train_loss=0.0278]

Epoch 198, global step 9950: 'train_loss' reached 0.02781 (best 0.02781), saving model to '/home/jupyter/work/resources/lightning_logs/version_14/checkpoints/epoch=198-step=9950.ckpt' as top 1


Epoch 199: |          | 50/? [00:06<00:00,  7.74it/s, v_num=14, train_loss=0.029] 

Epoch 199, global step 10000: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: |          | 50/? [00:06<00:00,  7.74it/s, v_num=14, train_loss=0.029]


In [9]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset_test, predictor=predictor, num_samples=100
)
forecasts = list(forecast_it)
targets = list(ts_it)
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

Running evaluation: 7it [00:00, 69.94it/s]
Running evaluation: 7it [00:00, 67.76it/s]
Running evaluation: 7it [00:00, 69.25it/s]
Running evaluation: 7it [00:00, 68.53it/s]
Running evaluation: 7it [00:00, 67.48it/s]
Running evaluation: 7it [00:00, 70.72it/s]
Running evaluation: 7it [00:00, 66.11it/s]
Running evaluation: 7it [00:00, 66.36it/s]
Running evaluation: 7it [00:00, 68.45it/s]
Running evaluation: 7it [00:00, 63.15it/s]
Running evaluation: 7it [00:00, 68.92it/s]
Running evaluation: 7it [00:00, 71.08it/s]
Running evaluation: 7it [00:00, 66.88it/s]
Running evaluation: 7it [00:00, 70.50it/s]
Running evaluation: 7it [00:00, 66.70it/s]
Running evaluation: 7it [00:00, 68.93it/s]
Running evaluation: 7it [00:00, 66.34it/s]
Running evaluation: 7it [00:00, 63.02it/s]
Running evaluation: 7it [00:00, 69.79it/s]
Running evaluation: 7it [00:00, 62.69it/s]
Running evaluation: 7it [00:00, 64.15it/s]
Running evaluation: 7it [00:00, 69.20it/s]
Running evaluation: 7it [00:00, 59.42it/s]
Running eva

In [10]:
print('Timegrad on regular dataset metrics:')
print("CRPS: {}".format(agg_metric["mean_wQuantileLoss"]))
print("ND: {}".format(agg_metric["ND"]))
print("NRMSE: {}".format(agg_metric["NRMSE"]))
print("MSE: {}".format(agg_metric["MSE"]))

print('')

print("CRPS-Sum: {}".format(agg_metric["m_sum_mean_wQuantileLoss"]))
print("ND-Sum: {}".format(agg_metric["m_sum_ND"]))
print("NRMSE-Sum: {}".format(agg_metric["m_sum_NRMSE"]))
print("MSE-Sum: {}".format(agg_metric["m_sum_MSE"]))

Timegrad on regular dataset metrics:
CRPS: 0.054710504611443216
ND: 0.0690697753085394
NRMSE: 0.6498842557863022
MSE: 241902.15123054018

CRPS-Sum: 0.02233674123229429
ND-Sum: 0.030012125785324334
NRMSE-Sum: 0.040314983705848954
MSE-Sum: 127439523.04761903


In [11]:
"""
with open('/home/jupyter/datasphere/project/models/electricity.pkl', 'wb') as f:
    pickle.dump(predictor, f)
"""

"\nwith open('/home/jupyter/datasphere/project/models/electricity.pkl', 'wb') as f:\n    pickle.dump(predictor, f)\n"

## Conduct Experiments

In [12]:
"""
# If you want to load already prepared model

transformation = estimator.create_transformation()
lightning_module = estimator.create_lightning_module()
predictor = estimator.create_predictor(transformation, lightning_module)
with open('/home/jupyter/work/resources/solar/model.pkl', 'rb') as f:
    predictor = pickle.load(f)
"""

"\n# If you want to load already prepared model\n\ntransformation = estimator.create_transformation()\nlightning_module = estimator.create_lightning_module()\npredictor = estimator.create_predictor(transformation, lightning_module)\nwith open('/home/jupyter/work/resources/solar/model.pkl', 'rb') as f:\n    predictor = pickle.load(f)\n"

In [13]:
def interpolate_points(dataset, num_points: int = 5):
    new_dataset = []
    with Pool(processes=cpu_count()) as pool:
        results = pool.starmap(interpolate_np_array, [(ts['target'], num_points) for ts in dataset])
        for i, ts in enumerate(dataset):
            mask = np.isnan(ts['target'])
            new_dataset.append({
                'target': ts['target'] * (1. - mask) + results[i] * mask,
                'start': ts['start'],
                'feat_static_cat': ts['feat_static_cat']
            })
    return new_dataset

In [ ]:
crps_sum_results = pd.DataFrame(columns=range(10))

with tqdm(total=9*10) as pbar:
    for p in np.linspace(0.1, 1, 9, endpoint=False): # frac of removed points
        dataset_test_p = omit_points(dataset_test, p)
        results_p = []

        # No interpolation
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset_test_p, 
            predictor=predictor, 
            num_samples=100,
        )
        forecasts = list(forecast_it)
        # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
        agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
        results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
        pbar.update(1)

        # Interpolation with num_points
        for num_points in range(1, 10):
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=interpolate_points(dataset_test_p, num_points), 
                predictor=predictor, 
                num_samples=100,
            )
            forecasts = list(forecast_it)
            # targets = list(ts_it)  # Do not uncomment, we need targets without NaNs, so we use ones calculated above.
            agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
            results_p.append(agg_metric['m_sum_mean_wQuantileLoss'])
            pbar.update(1)

        crps_sum_results.loc[p] = results_p

In [20]:
crps_sum_results

,0,1,2,3,4,5,6,7,8,9
0.1,0.027086,0.027214,0.026956,0.026900,0.027509,0.027209,0.027129,0.026895,0.027410,0.027262
0.2,0.028681,0.028619,0.028953,0.028961,0.028329,0.028718,0.029221,0.028905,0.028679,0.028268
0.3,0.040260,0.041241,0.040710,0.040827,0.040772,0.040829,0.040282,0.040518,0.040399,0.040977
0.4,0.049984,0.049600,0.049526,0.049539,0.048798,0.049651,0.049160,0.049208,0.049556,0.049113
0.5,0.218451,0.217940,0.218166,0.218247,0.218441,0.218621,0.218975,0.218295,0.218798,0.217990
0.6,0.405616,0.403999,0.405476,0.404888,0.404395,0.405973,0.405904,0.405156,0.404947,0.405183
0.7,0.365148,0.363907,0.364438,0.364306,0.363750,0.363791,0.363968,0.363736,0.364003,0.364552
0.8,0.380127,0.380777,0.380521,0.380265,0.379935,0.380647,0.380733,0.380244,0.380624,0.380668
0.9,0.318306,0.317880,0.316428,0.317360,0.319050,0.319102,0.318317,0.316586,0.315723,0.320595
